# Forces 🏋️

- [ForceGraph Top level API](https://github.com/vasturiano/force-graph#force-engine-d3-force-configuration)
  - Forcing Functions:
    - `d3Force(str, [fn])`
    - Default forces: {"link", "charge", "center"}
    - Example of
      ["collide" and "box"](https://vasturiano.github.io/force-graph/example/collision-detection/)([source](https://github.com/vasturiano/force-graph/blob/master/example/collision-detection/index.html))
    - https://www.d3indepth.com/force-layout/
      - forceCenter
      - forceCollide
        - [using mouse](https://observablehq.com/@d3/collision-detection/2?collection=@d3/d3-force)
      - forceLink
      - forceManyBody
      - forceRadial
      - forceX
      - forceY
    - [full on custom](https://technology.amis.nl/frontend/introduction-to-d3-force-for-simulation-and-animation/)
      - use handlebar templates?
  - [DAG](https://github.com/vasturiano/force-graph/blob/master/example/tree/index.html)

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import asyncio
import random

import ipywidgets as W
import traitlets as T

import ipyforcegraph.forces as F
from ipyforcegraph.utils import wait_for_change

In [ ]:
with __import__("importnb").Notebook():
    import Behaviors as B
    import Utils as U

In [ ]:
if __name__ == "__main__":
    fg, box = U.make_a_demo()
    B.add_graph_data(fg, box)
    graph_data = box.behaviors["graph_data"]
    display(box)

In [ ]:
force_link = F.LinkForce()
force_charge = F.ManyBodyForce()
force_center = F.CenterForce()
force_collide = F.CollisionForce(radius="4")
force_radial = F.RadialForce(radius="100", strength="0")

sim_forces = F.GraphForcesBehavior(
    forces={
        "link": force_link,
        "charge": force_charge,
        "center": force_center,
        "collide": force_collide,
        "radial_force": force_radial,
    }
)
fg.behaviors = [sim_forces]

In [ ]:
# TODO sometimes requires manually clicking capture first from the graph_data accoridan ui...
async def get_center():
    graph_data.capturing = True
    nodes = await wait_for_change(graph_data.sources[0], "nodes")
    return nodes.x.mean(), nodes.y.mean()


task = asyncio.ensure_future(get_center())
task

In [ ]:
graph_data.sources[0]

In [ ]:
graph_data.capturing

In [ ]:
radial_radius_slider = W.FloatSlider(description="Radius", min=0, value=10, max=200)
radial_strength_slider = W.FloatSlider(description="Strength", min=0, value=0, max=5)
T.dlink((radial_radius_slider, "value"), (force_radial, "radius"), str)
T.dlink((radial_strength_slider, "value"), (force_radial, "strength"), str)

W.VBox(
    [
        radial_radius_slider,
        radial_strength_slider,
    ]
)

In [ ]:
task

In [ ]:
charge_slider = W.FloatSlider(description="Charge", min=-50, value=-30, max=10)
T.dlink((charge_slider, "value"), (force_charge, "strength"), str)

charge_slider

In [ ]:
center_sliders = dict(
    x=W.FloatSlider(description="X", min=-200, max=200),
    y=W.FloatSlider(description="Y", min=-200, max=200),
    z=W.FloatSlider(description="Z", min=-200, max=200),
)

for key in ["x", "y", "z"]:
    T.link((center_sliders[key], "value"), (force_center, key))
W.VBox(list(center_sliders.values()))

In [ ]:
radius_slider = W.FloatSlider(description="Radius", min=-100)

T.dlink((radius_slider, "value"), (force_collide, "radius"), str)
radius_slider

In [ ]:
radius_slider.value = 40

In [ ]:
force_link = F.LinkForce()
force_charge = F.ManyBodyForce()
force_center = F.CenterForce(x=250)

sim_forces = F.GraphForcesBehavior(
    forces={
        "link": force_link,
        "charge": force_charge,
        "center": force_center,
        "x": None,
    }
)
fg.behaviors = [sim_forces]

In [ ]:
y = F.YForce()

sim_forces = F.GraphForcesBehavior(
    forces={
        "y": y,
    }
)
fg.behaviors = [sim_forces]

In [ ]:
y_slider = W.FloatSlider(description="Charge", min=-200, value=0, max=200)
T.dlink((y_slider, "value"), (y, "y"))

y_slider

In [ ]:
force_link = F.LinkForce()
force_charge = F.ManyBodyForce()
force_center = F.CenterForce(z=2000)

sim_forces = F.GraphForcesBehavior(
    forces={
        "link": force_link,
        "charge": force_charge,
        "center": force_center,
        "radial": None,
    }
)
fg.behaviors = [sim_forces]

## ticks

In [ ]:
warmup_ticks = W.IntSlider(description="warmup", min=0, max=100)
cooldown_ticks = W.IntSlider(description="cooldown", min=-1, max=10000)

T.link(
    (sim_forces, "warmup_ticks"),
    (warmup_ticks, "value"),
)
T.link((sim_forces, "cooldown_ticks"), (cooldown_ticks, "value"))
display(warmup_ticks, cooldown_ticks)

## alpha/velocity

In [ ]:
alpha_min = W.FloatSlider(
    description="alpha min", min=0, max=1, step=0.0001, readout_format=".4f"
)
alpha_decay = W.FloatSlider(
    description="alpha decay", min=0, max=1, step=0.0001, readout_format=".4f"
)
velocity_decay = W.FloatSlider(
    description="velocity_ decay", min=0, max=1, step=0.0001, readout_format=".4f"
)

T.link((sim_forces, "alpha_decay"), (alpha_decay, "value"))
T.link((sim_forces, "alpha_min"), (alpha_min, "value"))
T.link((sim_forces, "velocity_decay"), (velocity_decay, "value"))
display(alpha_min, alpha_decay, velocity_decay)